# Linear Regression

In [ ]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
# this allows plots to appear directly in the notebook
%matplotlib inline

## Example: Advertising Data

Let's take a look at some data, ask some questions about that data, and then use linear regression to answer those questions!

In [ ]:
# read data into a DataFrame
data = pd.read_csv('Advertising.csv', index_col=0)
data.head()

What are the **features**?
- TV: advertising dollars spent on TV for a single product in a given market (in thousands of dollars)
- Radio: advertising dollars spent on Radio
- Newspaper: advertising dollars spent on Newspaper

What is the **response**?
- Sales: sales of a single product in a given market (in thousands of widgets)

In [ ]:
# print the shape of the DataFrame
data.shape

There are 200 **observations**, and thus 200 markets in the dataset.

In [ ]:
# visualize the relationship between the features and the response using scatterplots
fig, axs = plt.subplots(1, 3, sharey=True)
data.plot(kind='scatter', x='TV', y='Sales', ax=axs[0], figsize=(16, 8))
data.plot(kind='scatter', x='Radio', y='Sales', ax=axs[1])
data.plot(kind='scatter', x='Newspaper', y='Sales', ax=axs[2])

## Questions About the Advertising Data

Let's pretend you work for the company that manufactures and markets this widget. The company might ask you the following: On the basis of this data, how should we spend our advertising money in the future?

This general question might lead you to more specific questions:
1. Is there a relationship between ads and sales?
2. How strong is that relationship?
3. Which ad types contribute to sales?
4. What is the effect of each ad type of sales?
5. Given ad spending in a particular market, can sales be predicted?

We will explore these questions below!

## Simple Linear Regression

Simple linear regression is an approach for predicting a **quantitative response** using a **single feature** (or "predictor" or "input variable"). It takes the following form:

$y = \beta_0 + \beta_1x$

What does each term represent?
- $y$ is the response
- $x$ is the feature
- $\beta_0$ is the intercept
- $\beta_1$ is the coefficient for x

Together, $\beta_0$ and $\beta_1$ are called the **model coefficients**. To create your model, you must "learn" the values of these coefficients. And once we've learned these coefficients, we can use the model to predict Sales!

## Estimating ("Learning") Model Coefficients

Generally speaking, coefficients are estimated using the **least squares criterion**, which means we are find the line (mathematically) which minimizes the **sum of squared residuals** (or "sum of squared errors"):

<img src="08_estimating_coefficients.png">

What elements are present in the diagram?
- The black dots are the **observed values** of x and y.
- The blue line is our **least squares line**.
- The red lines are the **residuals**, which are the distances between the observed values and the least squares line.

How do the model coefficients relate to the least squares line?
- $\beta_0$ is the **intercept** (the value of $y$ when $x$=0)
- $\beta_1$ is the **slope** (the change in $y$ divided by change in $x$)

Here is a graphical depiction of those calculations:

<img src="08_slope_intercept.png">

Let's use **Scikit-learn** to estimate the model coefficients for the advertising data:

In [ ]:
# create X and y
feature_cols = ['TV']
X = data[feature_cols]
y = data.Sales

# instantiate and fit
linreg = LinearRegression()
linreg.fit(X, y)

# print the coefficients
print(linreg.intercept_)
print(linreg.coef_)

## Interpreting Model Coefficients

How do we interpret the TV coefficient ($\beta_1$)?
- A "unit" increase in TV ad spending is **associated with** a 0.047537 "unit" increase in Sales.
- Or more clearly: An additional $1,000 spent on TV ads is **associated with** an increase in sales of 47.537 widgets.

Note that if an increase in TV ad spending was associated with a **decrease** in sales, $\beta_1$ would be **negative**.

## Using the Model for Prediction

Let's say that there was a new market where the TV advertising spend was **$50,000**. What would we predict for the Sales in that market?

$$y = \beta_0 + \beta_1x$$
$$y = 7.032594 + 0.047537 \times 50$$

In [ ]:
# manually calculate the prediction
7.032594 + 0.047537*50

Thus, we would predict Sales of **9,409 widgets** in that market.

Of course, we can also use Scikit-learn to make the prediction:

In [ ]:
linreg.predict([[50]])

## Plotting the Least Squares Line

Let's make predictions for the **smallest and largest observed values of x**, and then use the predicted values to plot the least squares line:

In [ ]:
# create a DataFrame with the minimum and maximum values of TV
X_new = pd.DataFrame({'TV': [data.TV.min(), data.TV.max()]})
X_new.head()

In [ ]:
# make predictions for those x values and store them
preds = linreg.predict(X_new)
preds

In [ ]:
# first, plot the observed data
data.plot(kind='scatter', x='TV', y='Sales')

# then, plot the least squares line
plt.plot(X_new, preds, c='red', linewidth=2)

## How Well Does the Model Fit the data?

The most common way to evaluate the overall fit of a linear model is by the **R-squared** value. R-squared is the **proportion of variance explained**, meaning the proportion of variance in the observed data that is explained by the model, or the reduction in error over the **null model**. (The null model just predicts the mean of the observed response, and thus it has an intercept and no slope.)

R-squared is between 0 and 1, and higher is better because it means that more variance is explained by the model. Here's an example of what R-squared "looks like":

<img src="08_r_squared.png">

You can see that the **blue line** explains some of the variance in the data (R-squared=0.54), the **green line** explains more of the variance (R-squared=0.64), and the **red line** fits the training data even further (R-squared=0.66). (Does the red line look like it's overfitting?)

Let's calculate the R-squared value for our simple linear model:

In [ ]:
# calculate the R-squared value for the model (by default it uses the R-squared score)
linreg.score(X, y)

**Is that a "good" R-squared value?**

- It's hard to say
- The threshold for a good R-squared value depends widely on the domain
- Therefore, it's most useful as a tool for **comparing different models**

# Multiple Linear Regression

Simple linear regression can easily be extended to include multiple features, which is called **multiple linear regression**:

$y = \beta_0 + \beta_1x_1 + ... + \beta_nx_n$

Each $x$ represents a different feature, and each feature has its own coefficient:

$y = \beta_0 + \beta_1 \times TV + \beta_2 \times Radio + \beta_3 \times Newspaper$

Let's estimate these coefficients:

In [ ]:
# create X and y
feature_cols = ['TV', 'Radio', 'Newspaper']
X = data[feature_cols]
y = data.Sales

# instantiate and fit
linreg2 = LinearRegression()
linreg2.fit(X, y)

# print the coefficients
print(linreg2.intercept_)
print(linreg2.coef_)

In [ ]:
# pair the feature names with the coefficients
list(zip(feature_cols, linreg2.coef_))

- For a given amount of Radio and Newspaper spending, an increase of \$1000 in **TV** spending is associated with an **increase in Sales of 45.8 widgets**
- For a given amount of TV and Newspaper spending, an increase of \$1000 in **Radio** spending is associated with an **increase in Sales of 188.5 widgets**.
- For a given amount of TV and Radio spending, an increase of \$1000 in **Newspaper** spending is associated with an **decrease in Sales of 1.0 widgets**. How could that be?

In [ ]:
# predict for a new observation
linreg2.predict([[100, 25, 25]])

In [ ]:
# calculate the R-squared
linreg2.score(X, y)

**Issure with R-squared**

- **R-squared will always increase as you add more features to the model**, even if they are unrelated to the response
    - Selecting the model with the highest R-squared is not a reliable approach for choosing the best linear model.

**Solution**

- **Adjusted R-squared**
    - Penalizes model complexity (to control for overfitting), but it generally under-penalizes complexity.

**Better Solution**

- **Train/test split or cross-validation**
    - More reliable estimate of out-of-sample error
    - Better for choosing which of your models will best generalize to out-of-sample data
    - There is extensive functionality for cross-validation in scikit-learn, including automated methods for searching different sets of parameters and different models
    - Importantly, cross-validation can be applied to any model, whereas the methods described above only apply to linear models

# Evaluation metrics for regression problems

Evaluation metrics for classification problems, such as **accuracy**, are not useful for regression problems. We need evaluation metrics designed for comparing **continuous values**.

Let's create some example numeric predictions, and calculate three common evaluation metrics for regression problems:
#### Define true and predicted response values

In [ ]:
y_true = [100, 50, 30, 20]
y_pred = [90, 50, 50, 30]

**Mean Absolute Error** (MAE) is the mean of the absolute value of the errors:
$$\frac 1n\sum_{i=1}^n|y_i-\hat{y}_i|$$
**Mean Squared Error** (MSE) is the mean of the squared errors:
$$\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2$$
 **Root Mean Squared Error** (RMSE) is the square root of the mean of the squared errors:
$$\sqrt{\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2}$$

In [ ]:
# calculate MAE, MSE, RMSE
print('Mean Absolute Error: ', metrics.mean_absolute_error(y_true, y_pred))
print('Mean Squared Error: ', metrics.mean_squared_error(y_true, y_pred))
print('Root Mean Squared Error: ', np.sqrt(metrics.mean_squared_error(y_true, y_pred)))

## Comparing these metrics:
- **MAE** is the easiest to understand, because it's the average error.
- **MSE** is more popular than MAE, because MSE "punishes" larger errors, which tends to be useful in the real world.
- **RMSE** is even more popular than MSE, because RMSE is interpretable in the "y" units.

#### All of these are **loss functions**, because we want to minimize them.

Here's an additional example, to demonstrate how MSE/RMSE punish larger errors:

#### Same true values as above

In [ ]:
y_true = [100, 50, 30, 20]

#### New set of predicted values

In [ ]:
y_pred = [60, 50, 30, 20]

#### MAE is the same as before

In [ ]:
print(metrics.mean_absolute_error(y_true, y_pred))

#### RMSE is larger than before

In [ ]:
print(np.sqrt(metrics.mean_squared_error(y_true, y_pred)))

# Using train/test split (or cross-validation)

### A better approach to feature selection!
- They attempt to directly estimate how well your model will **generalize** to out-of-sample data.
- They rely on **fewer assumptions** that linear regression.
- They can easily be applied to **any model**, not just linear models.

### define a function that accepts X and y and computes testing RMSE

In [ ]:
def train_test_rmse(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    return np.sqrt(metrics.mean_squared_error(y_test, y_pred))

### Let's try with all the features

In [ ]:
feature_cols = ['TV', 'Radio', 'Newspaper']
X = data[feature_cols]
train_test_rmse(X, y)

**Hmm, is there any chance that we can lower the RMSE value?** We can try exclude the high correlated features. Let's try to see which ones are correlated the least:

In [ ]:
# import Seaborn visualization library
import seaborn as sns

# plot the heatmap
sns.heatmap(data.corr(), annot=True)

### Exclude Newspaper

In [ ]:
feature_cols = ['TV', 'Radio']
X = data[feature_cols]
train_test_rmse(X, y)

**Voilá! A lower RMSE as expected!**

# Comparing linear regression with other models

## Advantages of linear regression:
- Simple to explain
- Highly interpretable
- Model training and prediction are fast
- No tuning is required (excluding regularization)
- Features don't need scaling
- Can perform well with a small number of observations

## Disadvantages of linear regression:
- Presumes a linear relationship between the features and the response
- Performance is (generally) not competitive with the best supervised learning methods due to high bias
- Sensitive to irrelevant features
- Can't automatically learn feature interactions